In [1]:
import os
import glob
import json
import quickstats
import sys, numpy as np
from quickstats.components import AnalysisBase

Welcome to JupyROOT 6.24/06

RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



In [2]:
# make sure hh combination fw path is defined
hh_comb_fw_path = os.environ.get('hh_combination_fw_path', None)

assert hh_comb_fw_path

gen_command = True

In [3]:
from quickstats.maths.numerics import is_integer, pretty_value
from pprint import pprint
def create_job(string, key):
    tmp = string.split(',')
    poi1 = tmp[0].split('=')[0]
    poi2 = tmp[1].split('=')[0]
    poi1_tmp = tmp[0].split('=')[1].split('_')
    poi2_tmp = tmp[1].split('=')[1].split('_')
    poi1_range = [float(p) for p in poi1_tmp[0:2]]
    poi1_step = float(poi1_tmp[2])
    poi2_range = [float(p) for p in poi2_tmp[0:2]]
    poi2_step = float(poi2_tmp[2])

    poi1_nsteps = int((poi1_range[1] - poi1_range[0]) / poi1_step)
    poi2_nsteps = int((poi2_range[1] - poi2_range[0]) / poi2_step)

    # slice poi1 with full poi2 range
    if poi1_nsteps < poi2_nsteps and poi2_nsteps > 12:
        poi1, poi1_range, poi1_step, poi1_nsteps, poi2, poi2_range, poi2_step, poi2_nsteps = poi2, poi2_range, poi2_step, poi2_nsteps, poi1, poi1_range, poi1_step, poi1_nsteps
        
    poi_slice_up = np.arange(poi1_range[0], poi1_range[1], poi1_step).round(decimals=2)
    new_dict = {}
    for i in range(poi1_nsteps):
        key_name = key+f'_job{i+1}'
        value = f"^{poi1}={poi_slice_up[i]}_{pretty_value(poi_slice_up[i]+poi1_step)}_{poi1_step},{poi2}={pretty_value(poi2_range[0])}_{pretty_value(poi2_range[1])}_{poi2_step}^"
        new_dict[key_name] = value
    return new_dict


In [9]:
def return_combine_command(paths, config):
    options = {
        "input_dir"    : paths['output'],
        "resonant_type": config['resonant_type'],
        "channels"     : ",".join(config['channels']),
        "file_expr"    : f"\"{config['file_expr']}\"",
        "config"       : paths['task_options'],
        "scheme"       : paths['correlation_schemes'],
        "parallel"     : config['parallel'],
        "skip-limit"   : ""
    }
    if config["blind"]:
        options["blind"] = ""
    else:
        options["unblind"] = ""
    if config["experimental"]:
        options["experimental"] = ""
    else:
        options["official"] = ""
    if config['cache']:
        options['cache'] = ""
    else:
        options['no-cache'] = ""
    command_str = "HHComb combine_ws " + " ".join([f"--{key} {value}" for key, value in options.items()])
    print(command_str)
    return command_str



# Step 1: SM point

In [10]:
# specify the input timestamp to use
timestamp = "20220520_noSgHparam"
withbr = None

paths = {
    'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
    'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022",
                                        f"nonres_SM.yaml"),
    'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", 
                                        "nonres_kl_v12.json"),
    'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}")
}

config = {
    'resonant_type': 'nonres',
    'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
    'file_expr'    : '<mass[F]>',
    'blind'        : False,
    'cache'        : True,
    'experimental' : True,
    'parallel'     : -1,
    'type'         : 'mu'
}
options = {
    "input_dir"    : paths['input'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "outdir"       : paths['output'],
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "parallel"     : config['parallel'],
    "skip-limit"   : "",
}

## process channels

In [11]:

if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] = ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] : ""
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""
    
    
print(options)
command_str1 = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str1)

{'input_dir': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220520_noSgHparam', 'resonant_type': 'nonres', 'channels': 'bbbb,bbtautau,bbyy', 'outdir': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam', 'file_expr': '"<mass[F]>"', 'config': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_SM.yaml', 'parallel': -1, 'skip-limit': '', 'unblind': '', 'experimental': '', 'cache': ''}
HHComb process_channels --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220520_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam --file_expr "<mass[F]>" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_comb

## combine ws

In [12]:
command_str = return_combine_command(paths, config)

HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_SM.yaml --scheme /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v12.json --parallel -1 --skip-limit  --unblind  --experimental  --cache 


## obtain path

In [8]:
rescaled_ws_paths = {}
for channel in config['channels']:
    ws_path = os.path.join(paths['output'], 'rescaled', config['resonant_type'], channel, "0.root")
    rescaled_ws_paths[channel]  = ws_path
# rescaled workspace path for combined workspacee
ws_path = glob.glob(os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0.root"))[0]
rescaled_ws_paths["combined"] = ws_path
rescaled_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbbb/0.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbtautau/0.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbyy/0.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0.root'}

## Step 2: Cross section scans

In [13]:
# specify the input timestamp to use
timestamp = "20220520_noSgHparam"
withbr = None

paths = {
    'input'              : os.path.join(hh_comb_fw_path, "FullRun2Workspaces", "original", "HHH2022", timestamp),
    'task_options'       : os.path.join(hh_comb_fw_path, "configs", "task_options"       , "HHH2022",
                                        f"nonres_kl_kt_xsection.yaml"),
    'correlation_schemes': os.path.join(hh_comb_fw_path, "configs", "correlation_schemes", "HHH2022", 
                                        "nonres_kl_v12.json"),
    'output'             : os.path.join(os.getcwd(), f"outputs_HHH2022_{timestamp}")
}

config = {
    'resonant_type': 'nonres',
    'channels'     : ['bbbb', 'bbtautau', 'bbyy'],
    'file_expr'    : '<mass[F]>_kl',
    'blind'        : False,
    'cache'        : True,
    'experimental' : True,
    'parallel'     : -1,
    'type'         : 'xsec'
}
options = {
    "input_dir"    : paths['input'],
    "resonant_type": config['resonant_type'],
    "channels"     : ",".join(config['channels']),
    "outdir"       : paths['output'],
    "file_expr"    : f"\"{config['file_expr']}\"",
    "config"       : paths['task_options'],
    "parallel"     : config['parallel'],
    "skip-limit"   : "",
}

if config["blind"]:
    options["blind"] = ""
else:
    options["unblind"] = ""
if config["experimental"]:
    options["experimental"] = ""
else:
    options["official"] = ""
if config['cache']:
    options['cache'] = ""
else:
    options['no-cache'] = ""
command_str1 = "HHComb process_channels " + " ".join([f"--{key} {value}" for key, value in options.items()])
print(command_str1)

if config['type'] == 'xsec':
    options['fix'] = f"\"THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,alpha_THEO_XS_PDFalphas_ggFSMHH*=0,alpha_THEO_XS_SCALEMTop_ggFSMHH*=0,THEO_XS_COMBINED_HH_ggF*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_PDFalphas_HH_ggF*=0,THEO_XS_SCALE_HH_VBF*=0\""


HHComb process_channels --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/HHH2022/20220520_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_xsection.yaml --parallel -1 --skip-limit  --unblind  --experimental  --cache 


In [14]:
command_str = return_combine_command(paths, config)

HHComb combine_ws --input_dir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam --resonant_type nonres --channels bbbb,bbtautau,bbyy --file_expr "<mass[F]>_kl" --config /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/task_options/HHH2022/nonres_kl_kt_xsection.yaml --scheme /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/configs/correlation_schemes/HHH2022/nonres_kl_v12.json --parallel -1 --skip-limit  --unblind  --experimental  --cache 


In [15]:
rescaled_ws_paths = {}
for channel in config['channels']:
    ws_path = os.path.join(paths['output'], 'rescaled', config['resonant_type'], channel, "0_kl.root")
    rescaled_ws_paths[channel] = ws_path
# rescaled workspace path for combined workspacee
ws_path = glob.glob(os.path.join(paths['output'], 'combined', config['resonant_type'], "*", "0_kl.root"))[0]
rescaled_ws_paths["combined"] = ws_path
rescaled_ws_paths

{'bbbb': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbbb/0_kl.root',
 'bbtautau': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbtautau/0_kl.root',
 'bbyy': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbyy/0_kl.root',
 'combined': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/combined/nonres/A-bbbb_bbtautau_bbyy-fullcorr/0_kl.root'}

In [16]:
# use '_job' + suffix to split scan range
param_expr_maps = {
    'klambda_job1': "^klambda=-15_0_0.2^",
    'klambda_job2': "^klambda=0_20_0.2^",
    'klambda_final': "^klambda=-15_20_0.2^",
#     'k2v': "^k2V=-2_4_0.2^",
#     'klambdak2v_final': "^klambda=-15_20_0.2,k2V=-2_4_0.2^",
#     'k2vkv_final': "^kV=-3_3_0.2,k2V=-2_6_0.2^",
}

# param_expr_maps.update(create_job('klambda=-15_20_0.2,k2V=-2_4_0.2', 'klambdak2v'))
# param_expr_maps.update(create_job('kV=-3_3_0.2,k2V=-2_6_0.2', 'k2vkv'))
pprint(param_expr_maps)

{'klambda_final': '^klambda=-15_20_0.2^',
 'klambda_job1': '^klambda=-15_0_0.2^',
 'klambda_job2': '^klambda=0_20_0.2^'}


In [17]:
options = {
    'poi'         : 'xsec_br',
    'data'        : 'combData',
    'unblind'     : '',
    'fix'         : "^THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,alpha_THEO_XS_PDFalphas_ggFSMHH*=0,alpha_THEO_XS_SCALEMTop_ggFSMHH*=0,THEO_XS_COMBINED_HH_ggF*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_PDFalphas_HH_ggF*=0,THEO_XS_SCALE_HH_VBF*=0^",
}
command_str_map = {}

condor = False
if condor:
    original_stdout = sys.stdout
    sys.stdout = open('job2.txt', 'w')

for channel in rescaled_ws_paths:
    print(f"# Channel: {channel}")
    input_path  = rescaled_ws_paths[channel]
    for key, expr in param_expr_maps.items():
        
        if condor and 'final' in key:
            continue
        if not condor and 'job' in key:
            continue
        if 'final' in key:
            outdir = os.path.join(paths['output'], 'xsection_scan', channel, key.replace('_final', ''))
        else:
            outdir = os.path.join(paths['output'], 'xsection_scan', channel, (key+'1')[:key.find('_job')])

        channel_options = {"input_path": input_path, "outdir": outdir, "param_expr": expr, **options}
        channel_command_str = "quickstats limit_scan " + \
                              " ".join([f"--{key} {value}" for key, value in channel_options.items()])
        command_str_map[channel] = channel_command_str


        if not condor:
            print(channel_command_str.replace('^', '"'))
        else:
            print("Arguments =", channel_command_str.replace(' ', '____'))
            print("Queue 1")
    print()

if condor: 
    sys.stdout = original_stdout

# Channel: bbbb
quickstats limit_scan --input_path /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbbb/0_kl.root --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/xsection_scan/bbbb/klambda --param_expr "klambda=-15_20_0.2" --poi xsec_br --data combData --unblind  --fix "THEO_XS_fixmu_*=0,alpha_THEO_XS_PDFalphas_VBFSMHH*=0,alpha_THEO_XS_PDFalphas_ggFSMHH*=0,alpha_THEO_XS_SCALEMTop_ggFSMHH*=0,THEO_XS_COMBINED_HH_ggF*=0,THEO_XS_PDFalphas_HH_VBF*=0,THEO_XS_PDFalphas_HH_ggF*=0,THEO_XS_SCALE_HH_VBF*=0"

# Channel: bbtautau
quickstats limit_scan --input_path /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_HHH2022_20220520_noSgHparam/rescaled/nonres/bbtautau/0_kl.root --outdir /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/tutorials/HHH2022/outputs_H